# Complete Amazon Product Scrape with Multiple Sellers 
**Overview:**
In this project, I developed a web scraper using Python's Requests module to scrape complete product information from Amazon, including data from multiple sellers. The scraper extracts more than 25 data parameters such as product title, prices, rating, description, brand, seller information , all_variants , specification, amazon_tags and more.

**Common abbreviations**
- xp : Xpath
- pp : Product Page
- sp : Seller Page

In [1]:
import requests
from lxml import html
import pandas as pd
import re
import json
import time
import random
from datetime import datetime

In [2]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    }

seller_page_headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
    }

seller_page_cookies = {
    'aws-session-id': '748-0359166-8309551',
    'i18n-prefs': 'USD',
    'session-token': '"gDKmkLOnV8DkP1muUN2ZhofLLMzmGQWIHg6gBMg5sg2XZheZxtSbtvFHEn2Lye3zUgl8H8zWZL0tqW+3j5aXemxOK/gN//IAovGVQ91Z9+Z7zK9Kc1Kn3etaxbdaZisZQ80UpiFprxrsa1UMzvYfpNpQSGogsIAeIj2rO94nmizlUHZILKjT4++mZBt3kZNcN/GfmtTcvzYlgaXDbbhzMsJoIclk2BdEqEOrmFO9X5A="',
}

In [3]:
# Temporary Headers & Cookies - US California 
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': 'aws-session-id=748-0359166-8309551; aws-analysis-id=748-0359166-8309551; _mkto_trk=id:112-TZM-766&token:_mch-amazon.com-1671558076882-14926; regStatus=pre-register; aws-session-id-time=1671558078l; session-id=145-8025491-1599610; session-id-time=2082787201l; i18n-prefs=USD; ubid-main=134-8949560-5287142; aws-target-data=%7B%22support%22%3A%221%22%7D; aws-target-visitor-id=1676548308253-168032.31_0; AMCV_7742037254C95E840A4C98A6%40AdobeOrg=1585540135%7CMCIDTS%7C19451%7CMCMID%7C17917486245895676673171748920370395691%7CMCAAMLH-1681139195%7C12%7CMCAAMB-1681139195%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1680541595s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.4.0; lc-main=en_US; session-token="JawTxHeevVzSlTPvQMNfgSV/3Ohvc8qLm4t4rnp/s3JixsAKu2NBkDYha5lS7Nww+eUdxJpmPHGXy7q8PiEXnCsjUU23B1QUWH2iQpnSlz4gfVu4qdYNuMjCvUVQ+jehufArpXhWx+O+spbMAjwr/wlFn2zZE6zGUscMb3SinE0H+dCVN2/kV+7mGh4Ca0YoWs7oncD4kj2UUo0B9/BbTj8X+FvLr4pujNVj3cTkaRE="; csm-hit=tb:FDS7D4ZSKTZ77QH8RGVE+s-4PXKTBE38X79XW1VX7NA|1681389811085&t:1681389811085&adb:adblk_yes',
    'device-memory': '8',
    'downlink': '5.35',
    'dpr': '1.1979166269302368',
    'ect': '4g',
    'referer': 'https://www.amazon.com/SAMSUNG-Smartphone-Unlocked-Brightest-Processor/dp/B09MVZSW5V/?th=1',
    'rtt': '200',
    'sec-ch-device-memory': '8',
    'sec-ch-dpr': '1.1979166269302368',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"15.0.0"',
    'sec-ch-viewport-width': '1017',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'viewport-width': '1017',
}

cookies = {
    'aws-session-id': '748-0359166-8309551',
    'aws-analysis-id': '748-0359166-8309551',
    '_mkto_trk': 'id:112-TZM-766&token:_mch-amazon.com-1671558076882-14926',
    'regStatus': 'pre-register',
    'aws-session-id-time': '1671558078l',
    'session-id': '145-8025491-1599610',
    'session-id-time': '2082787201l',
    'i18n-prefs': 'USD',
    'ubid-main': '134-8949560-5287142',
    'aws-target-data': '%7B%22support%22%3A%221%22%7D',
    'aws-target-visitor-id': '1676548308253-168032.31_0',
    'AMCV_7742037254C95E840A4C98A6%40AdobeOrg': '1585540135%7CMCIDTS%7C19451%7CMCMID%7C17917486245895676673171748920370395691%7CMCAAMLH-1681139195%7C12%7CMCAAMB-1681139195%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1680541595s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.4.0',
    'lc-main': 'en_US',
    'session-token': '"JawTxHeevVzSlTPvQMNfgSV/3Ohvc8qLm4t4rnp/s3JixsAKu2NBkDYha5lS7Nww+eUdxJpmPHGXy7q8PiEXnCsjUU23B1QUWH2iQpnSlz4gfVu4qdYNuMjCvUVQ+jehufArpXhWx+O+spbMAjwr/wlFn2zZE6zGUscMb3SinE0H+dCVN2/kV+7mGh4Ca0YoWs7oncD4kj2UUo0B9/BbTj8X+FvLr4pujNVj3cTkaRE="',
    'csm-hit': 'tb:FDS7D4ZSKTZ77QH8RGVE+s-4PXKTBE38X79XW1VX7NA|1681389811085&t:1681389811085&adb:adblk_yes',
}




## Product Page Data Scrape

In [4]:
# Ship and sold by amazon
url = 'https://www.amazon.com/dp/B0002RI2PG/?th=1'
# ship and sold by other seller
url = 'https://www.amazon.com/Sleeve-Blouses-Women-Tops-Dressy/dp/B0C241J4KC/'
url = 'https://www.amazon.com/Dell-XPS-9310-Touchscreen-Laptop/dp/B09WB6NVTT/'
## Digital card
url = 'https://www.amazon.com/PlayStation-Store-Gift-Card-Digital/dp/B00GAC1D2G/'
## OOS product
url = 'https://www.amazon.com/Apple-iPhone-Pro-128GB-Gold/dp/B0BYLLTCMB/'
# Lenovo IdeaPad all tags
url = 'https://www.amazon.com/Lenovo-IdeaPad-Pentium-Windows-Included/dp/B0BNLYG895/'
## Multivariant, Multiseller - SAMSUNG Galaxy S22 
url = 'https://www.amazon.com/SAMSUNG-Smartphone-Unlocked-Brightest-Processor/dp/B09MVZSW5V/'

In [5]:
res = requests.get(url, headers=headers, cookies=cookies)
tree = html.fromstring(res.text)
res.status_code

200

In [6]:
def extract_date():
    # For Timezone - import pytz library
    # time_now = datetime.datetime.now(tz=pytz.timezone('Asia/Dubai'))
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

print("Extract Date: ", extract_date())

Extract Date:  2023-04-18 19:47:39


In [7]:
def product_name():
    product_name_xp1 = tree.xpath("//span[@id='productTitle']/text()")
    
    product_name = ''
    if product_name_xp1:
        product_name = product_name_xp1[0].strip()
    
    return product_name

product_name()

'SAMSUNG Galaxy S22 Ultra Cell Phone, Factory Unlocked Android Smartphone, 128GB, 8K Camera & Video, Brightest Display Screen, S Pen, Long Battery Life, Fast 4nm Processor, US Version, Phantom Black'

In [8]:
def asin():
    asin_xp1 = tree.xpath("//input[@id='ASIN']/@value")
    asin_xp2 = tree.xpath("//input[@id='attach-baseAsin']/@value")

    asin = ""
    if asin_xp1:
        asin = asin_xp1[0].strip()
    elif asin_xp2:
        asin = asin_xp2[0].strip()
    return asin

asin()

'B09MVZSW5V'

In [9]:
def old_price():
    old_price_xp1 = tree.xpath("//div[@id='centerCol']//span[contains(text(), 'List Price')]//span[@class='a-offscreen']/text()")
    old_price_xp2 = tree.xpath("//div[@id='centerCol']//td[contains(text(), 'List Price')]//following::span[@class='a-offscreen'][1]/text()")
    old_price_xp3 = tree.xpath("//div[@id='centerCol']//span[contains(text(), 'M.R.P.')]//span[@class='a-offscreen']/text()")
    old_price_xp4 = tree.xpath("//div[@id='centerCol']//td[contains(text(), 'M.R.P.')]//following::span[@class='a-offscreen'][1]/text()")
    
    old_price_xp5 = tree.xpath("//div[@id='centerCol']//td[contains(text(), 'Was:')]//following::span[@class='a-offscreen'][1]/text()")

    old_price = ''
    if old_price_xp1:
        old_price = re.sub("[^0-9.]+", '', old_price_xp1[0])
    elif old_price_xp2:
        old_price = re.sub("[^0-9.]+", '', old_price_xp2[0])
    elif old_price_xp3:
        old_price = re.sub("[^0-9.]+", '', old_price_xp3[0])
    elif old_price_xp4:
        old_price = re.sub("[^0-9.]+", '', old_price_xp4[0])
    elif old_price_xp5:
        old_price = re.sub("[^0-9.]+", '', old_price_xp5[0])
    return old_price

print("Old Price: ", old_price())

Old Price:  1199.99


In [10]:
def new_price():
    new_price_xp1 = tree.xpath("//div[@id='centerCol']//span[contains(@class, 'apexPriceToPay')]//span[@class='a-offscreen']/text()")
    new_price_xp2 = tree.xpath("//div[@id='centerCol']//span[contains(@class, 'priceToPay')]//span[@class='a-offscreen']/text()")
    
    new_price = ''
    if new_price_xp1:
        new_price = re.sub("[^0-9.]+", '', new_price_xp1[0])
    elif new_price_xp2:
        new_price = re.sub("[^0-9.]+", '', new_price_xp2[0])
    return new_price

print("New Price: ", new_price())

New Price:  949.99


In [11]:
def seller_info_pp():
    seller_info = {}
    ship_from_pp_xp1 = tree.xpath("//div[@id='rightCol']//div[@tabular-attribute-name='Ships from']//span[contains(@class, 'tabular-buybox-text-message')]/text()")
    
    sold_by_pp_xp1 = tree.xpath("//div[@id='rightCol']//div[@tabular-attribute-name='Sold by']//span[contains(@class, 'tabular-buybox-text-message')]//a/text()")
    sold_by_pp_xp2 = tree.xpath("//div[@id='rightCol']//div[@tabular-attribute-name='Sold by']//span[contains(@class, 'tabular-buybox-text-message')]/text()")
    
    ship_from_pp = sold_by_pp = ''
    
    if ship_from_pp_xp1:
        seller_info['Ship from'] = ship_from_pp_xp1[0].strip()
    if sold_by_pp_xp1:
        seller_info['Sold by'] = sold_by_pp_xp1[0].strip()
    elif sold_by_pp_xp2:
        seller_info['Sold by'] = sold_by_pp_xp2[0].strip()
    
    return seller_info

print("Seller info: ", seller_info_pp())

Seller info:  {'Ship from': 'Amazon.com', 'Sold by': 'Amazon.com'}


In [12]:
def brand():
    brand_xp1 = tree.xpath("//div[@id='centerCol']//a[@id='bylineInfo']/text()")
    
    brand = ''
    if brand_xp1:
        brand = brand_xp1[0]
        brand = brand.replace('Visit the', '').replace('Store', '').replace('Brand:', '').strip()
    
    return brand

print("Brand: ", brand())

Brand:  SAMSUNG


In [13]:
def category_breadcrumbs():
    ''' Return: Category breadcrumbs list '''
    category_breadcrumbs_bb_xp = tree.xpath("//div[@id='wayfinding-breadcrumbs_container']//a[@class='a-link-normal a-color-tertiary']/text()")
    category_breadcrumbs = ''
    if category_breadcrumbs_bb_xp:
        category_breadcrumbs = ' > '.join(list(map(lambda x:x.strip(), category_breadcrumbs_bb_xp)))

    return category_breadcrumbs

print("Category Breadcrumbs: ", category_breadcrumbs())

Category Breadcrumbs:  Cell Phones & Accessories > Cell Phones


In [14]:
def stock_info():
    stock_info_xp1 = tree.xpath("//div[@id='rightCol']//div[@id='availability']//span[contains(@class, 'a-size-medium')]/text()")
    stock_info = ''
    
    if stock_info_xp1:
        stock_info = stock_info_xp1[0].strip()
    
    return stock_info

print("Stock info: ", stock_info())

Stock info:  In Stock


In [15]:
def number_of_offers():
    no_of_offers_xp1 = tree.xpath("//div[@id='centerCol']//span[@data-action='show-all-offers-display']//span[contains(text(), 'New')]/text()")
    no_of_offers_xp2 = tree.xpath("//div[@id='rightCol']//div[@class='olp-text-box']//span[contains(text(), 'New')]/text()")
    
    no_of_offers = ''
    if no_of_offers_xp1:
        no_of_offers = re.sub("[^0-9]+", '', no_of_offers_xp1[0])
    elif no_of_offers_xp2:
        no_of_offers = re.sub("[^0-9]+", '', no_of_offers_xp2[0])
        
    return no_of_offers

print("Number of offers: ", number_of_offers())

Number of offers:  14


In [16]:
def price_start_from():
    price_from_xp1 = tree.xpath("//div[@id='centerCol']//span[@data-action='show-all-offers-display']//span[contains(text(), 'New')]//following::span[@class='a-size-base a-color-price']/text()")
    price_from_xp2 = tree.xpath("//div[@id='rightCol']//div[contains(@class, 'olp-text-box')]//span[@class='a-offscreen']/text()")
    price_from_xp3 = tree.xpath("//div[@id='rightCol']//div[contains(@class, 'olp-text-box')]//span[@class='a-size-base a-color-base']/text()")

    price_from = ''
    if price_from_xp1:
        price_from = re.sub("[^0-9.]+", '', price_from_xp1[0])
    elif price_from_xp2:
        price_from = re.sub("[^0-9.]+", '', price_from_xp2[0])
    elif price_from_xp3:
        price_from = re.sub("[^0-9.]+", '', price_from_xp3[0])
    
    return price_from

print("Price starts from: ", price_start_from())

Price starts from:  637.60


In [17]:
def product_rating():
    product_rating_xp = tree.xpath("//span[@id='acrPopover']/@title")
    product_rating = ''
    if product_rating_xp:
        # '4.1 out of 5 stars'
        product_rating = re.sub("[^0-9.]+", '', product_rating_xp[0].split()[0])

    return product_rating

print("Product Rating: ", product_rating())

Product Rating:  4.6


In [18]:
def product_reviews():
    product_review_xp = tree.xpath("//span[@id='acrCustomerReviewText']/text()")
    product_review = ''
    if product_review_xp:
        #product_review = product_review_xp[0].strip().replace('ratings','').replace('rating','')
        product_review = re.sub("[^0-9]+", '', product_review_xp[0])

    return product_review

print("Product Review: ", product_reviews())

Product Review:  973


In [19]:
def images_url():
    image_url_xp = tree.xpath("//div[@id='altImages']//span[contains(@class, 'a-button-text')]/img[@src]/@src")
    image_url = ''
    if image_url_xp:
        image_url = ' | '.join(x.strip() for x in image_url_xp)

    return image_url

print("Images URL: ", images_url())

Images URL:  https://m.media-amazon.com/images/I/41Te8HXFyjL._AC_US40_.jpg | https://m.media-amazon.com/images/I/31p609sUKWL._AC_US40_.jpg | https://m.media-amazon.com/images/I/41yw5ZalmkL._AC_US40_.jpg | https://m.media-amazon.com/images/I/310d9kayKWL._AC_US40_.jpg | https://m.media-amazon.com/images/I/412jSijiWIL._AC_US40_.jpg | https://m.media-amazon.com/images/I/318dJhz5rpL._AC_US40_.jpg | https://m.media-amazon.com/images/I/41qhyNzaPzL.SS40_BG85,85,85_BR-120_PKdp-play-icon-overlay__.jpg | https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel._V192234675_.gif


In [20]:
def fbt_content():
    ''' Return: Frequently Bought Together products separated by || '''
    fbt_content1_xp = tree.xpath("//div[contains(@class, 'cardRoot bucket')]//span[@class='a-label a-checkbox-label']//a[@class='a-link-normal']//span/text()")

    fbt_content = ''
    if fbt_content1_xp:
        fbt_content = ' || '.join(x.strip() for x in fbt_content1_xp)

    return fbt_content

print("Frequently Bought Together products: ", fbt_content())

Frequently Bought Together products:  IMBZBK [3+3 Pack for Samsung Galaxy S22 Ultra 5G Screen Protector [Not Glass], 3 Pack Flexible TPU Film with 3 Pack Tempered Glass Camera Lens Protector, Fingerprint Compatible, Case Friendly || OtterBox COMMUTER SERIES Case for Galaxy S22 Ultra - BLACK


In [21]:
# Todo: Find new product and check xpath
def card_offer():
    prime_card_offer_xp = tree.xpath("//div[@id='centerCol']//div[@class='a-section maple-banner__text']//span/text()")
    prime_card_offer = ''
    
    if prime_card_offer_xp:
        prime_card_offer = ' '.join(list(map(lambda x: x.strip(), prime_card_offer_xp)))

    return prime_card_offer

print("Card offer: ", card_offer())

Card offer:  Pay $79.17/month for 12 months , interest-free upon approval for the Amazon Rewards Visa Card


In [22]:
def selected_variant():
    # Selected Variant xpath
    selected_variant_xp1 = tree.xpath("//div[@id='twister-plus-inline-twister']//span[contains(@class, 'a-size-base a-color-base inline-twister-dim-title-value a-text-bold')]/text()")
    # selected_variant_xp2 = tree.xpath("//div[@id='twisterContainer']//form/div//option[@selected]/text()")
    if selected_variant_xp1:
        selected_variant = ' || '.join(list(map(lambda x: x.strip(), selected_variant_xp1)))
        
    return selected_variant

print("Selected Variants: ", selected_variant())

Selected Variants:  Phantom Black || 128GB || S22 Ultra


In [23]:
def all_variants():
    # All variant xpath
    # Color
    all_variant_xp1 = tree.xpath("//div[@id='twister-plus-inline-twister']//img[contains(@class, 'swatch-image ')]/@alt")
    # Style, Size - p text
    all_variant_xp2 = tree.xpath("//div[@id='twister-plus-inline-twister']//span[@class='a-button-inner']//span[contains(@class, 'swatch-title-text')]/text()")
    
    if all_variant_xp1 and all_variant_xp2:
        all_variant_list = [x.strip() for x in all_variant_xp1] + [x.strip() for x in all_variant_xp2]
    elif all_variant_xp1:
        all_variant_list = [x.strip() for x in all_variant_xp1]
    elif all_variant_xp2:
        all_variant_list = [x.strip() for x in all_variant_xp2]
    else:
        all_variant_list = []
        
    if all_variant_list:
        all_variants = ' || '.join(list(map(lambda x: x.strip(), all_variant_list)))
    
    return all_variants

print("All Variants: ", all_variants())

All Variants:  Phantom Black || Burgundy || Green || Phantom White || Bora Purple || Pink Gold || 128GB || 256GB || 512GB || S22 Ultra || S22 || S22 +


In [24]:
def product_description():
    # Product Description
    description_xp1 = tree.xpath("//div[@id='productDescription']//span/text()")
    # About this item - Center col
    description_xp2 = tree.xpath("//div[@id='feature-bullets']//span[@class='a-list-item']/text()")
    description = ''
    if description_xp1 and description_xp2:
        description1 = description_xp1[0].strip()
        description2 = ' | '.join(list(filter(None, [x.strip() for x in description_xp2])))
        description = description1 + ' | ' + description2
    elif description_xp1:
        description = description_xp1[0].srip()
    elif description_xp2:
        description = ' | '.join(list(filter(None, [x.strip() for x in description_xp2])))

    return description

print("Product Description: ", product_description())

Product Description:  For Gen Z, video isn’t just video. They think of it as a lifestyle – a universal language for how they learn, grow, express, talk, shop, connect, create, and fight boredom with their friends in real time. That’s why they’re demanding a mobile device that finally breaks all the rules of video and makes their everyday more epic than ever. Introducing Samsung Galaxy S22 Ultra, the brilliant new smartphones designed specifically to break the rules of video to meet the needs of Gen Z’s native language; video. With this new video machine, we’re setting an epic new standard. Now, you can communicate with your friends online while you simultaneously co-watch the latest new YouTube videos.**** Record your greatest nights with breathtaking low-light video like never before. Capture the true colors and contrasts of late-night parties that couldn’t be properly depicted before. And you can finally devour vids in direct sunlight with a screen that’s so bright, you’ll never have

In [25]:
def specification():
    # Todo: If specification table not available take it form product details
    # Common table tag - th/td not contain Additional Info Table - ASIN, Customer Reviews, Best Sellers Rank, Date First Available
    specification_th_xp = tree.xpath("//table[not(contains(@id, 'productDetails_detailBullets_sections1'))]//th[contains(@class, 'prodDetSectionEntry')]/text()")
    specification_td_xp = tree.xpath("//table[not(contains(@id, 'productDetails_detailBullets_sections1'))]//td[contains(@class, 'prodDetAttrValue')]/text()")

    specification = {}
    if len(specification_th_xp)  == len(specification_td_xp):
        for item_header, item_data  in zip(specification_th_xp,specification_td_xp):
            specification_th_key = item_header.replace('\u200e', '').strip()
            specification_td_value = item_data.replace('\u200e', '').strip()

            specification[specification_th_key] = specification_td_value

    return specification

print("Specification: ", specification())

Specification:  {}


In [26]:
def additional_info_table():
    additional_info = {}
    # For looping rows
    additional_info_table_xp = tree.xpath("//table[@id='productDetails_detailBullets_sections1']//tr")
    
    if additional_info_table_xp:
        for row in range(1, len(additional_info_table_xp)+2):
            # Need to loop row wise for accurate info as table info not equal for tr and td tags 
            additional_info_th_xp = tree.xpath(f"//div[@id='prodDetails']//tr[{row}]//th[contains(@class, 'prodDetSectionEntry')]/text()")
            additional_info_td_xp = tree.xpath(f"//div[@id='prodDetails']//tr[{row}]//td[contains(@class, 'prodDetAttrValue')]/text()")
            additional_info_cust_star_td_xp = tree.xpath(f"//div[@id='prodDetails']//tr[{row}]//td[contains(@class, 'a-size-base')]//span[@class='a-icon-alt']//text()")
            additional_info_cust_rating_td_xp = tree.xpath(f"//div[@id='prodDetails']//tr[{row}]//td[contains(@class, 'a-size-base')]//span[@class='a-size-base']/text()")
            
            if additional_info_th_xp and additional_info_td_xp:
                # Data add to dictionary - Dynamic Key value
                additional_info[additional_info_th_xp[0].replace('\u200e', '').strip()] = additional_info_td_xp[0].replace('\u200e', '').strip()
            elif additional_info_th_xp and (additional_info_td_xp or (additional_info_cust_rating_td_xp and additional_info_cust_star_td_xp)):
                # Customer review special case outside td
                if additional_info_cust_star_td_xp and additional_info_cust_rating_td_xp:
                    cust_review = additional_info_cust_star_td_xp[0].strip() + ' || ' + additional_info_cust_rating_td_xp[0].strip()
                    
                    additional_info[additional_info_th_xp[0].replace('\u200e', '').strip()] = cust_review
    
    return additional_info

print("Additional_info: ", additional_info_table())

Additional_info:  {'Package Dimensions': '7.05 x 3.66 x 0.98 inches', 'Item Weight': '13 ounces', 'ASIN': 'B09MVZSW5V', 'Item model number': 'SM-S908UZKAXAA', 'Batteries': '1 Lithium Ion batteries required. (included)', 'Customer Reviews': '4.6 out of 5 stars || 973 ratings', 'Date First Available': 'February 9, 2022', 'Manufacturer': 'Samsung Electronincs', 'Country of Origin': 'Vietnam'}


In [27]:
def amazon_tags():
    amazon_choice_xp = tree.xpath("//div[@class='ac-badge-wrapper']//span[@class='a-size-small aok-float-left ac-badge-rectangle']//span/text()")
    best_seller_xp = tree.xpath("//div[@class='badge-wrapper']//i/text()")

    amazon_tag = ''
    if amazon_choice_xp and best_seller_xp:
        amazon_choice = ' '.join(x.strip() for x in amazon_choice_xp)
        best_seller = best_seller_xp[0].strip()
        amazon_tag = amazon_choice + ' || ' + best_seller
    elif amazon_choice_xp:
        amazon_tag = ' '.join(x.strip() for x in amazon_choice_xp)
    elif best_seller_xp:
        amazon_tag = best_seller_xp[0].strip()

    return amazon_tag

print("Amazon tags: ", amazon_tags())

Amazon tags:  Amazon's Choice


In [28]:
def product_input_url(url):
    ''' input url '''
    return url

print("Product Input Url: ", product_input_url(url))

Product Input Url:  https://www.amazon.com/SAMSUNG-Smartphone-Unlocked-Brightest-Processor/dp/B09MVZSW5V/


In [29]:
def product_output_url():
    ''' Get url from website while scraping'''
    product_output_url_xp = tree.xpath("//link[@rel='canonical']/@href")
    product_output_url = ''
    if product_output_url_xp:
        product_output_url = product_output_url_xp[0].strip()

    return product_output_url

print("Product Output Url: ", product_output_url())

Product Output Url:  https://www.amazon.com/SAMSUNG-Smartphone-Unlocked-Brightest-Processor/dp/B09MVZSW5V


In [30]:
def warranty():
    # Although it has warranty, Amazon rarely provides it. Capturing for some condition where it shows warranty.  
    warranty_xp = tree.xpath("//div[@class='a-column a-span6']//a[contains(text(),  'Warranty')]/text()")
    warranty = ''
    if warranty_xp:
        warranty = 'Yes'

    return warranty

print("Warranty: ", warranty())

Warranty:  


### Product DataFrame

In [31]:
product_data = [{
    'extract_date' : extract_date(),
    'product_name' : product_name(),
    'asin' : asin(),
    'old_price' : old_price(),
    'new_price' : new_price(),
    'seller_info_pp' : seller_info_pp(),
    'brand' : brand(),
    'category_breadcrumbs' : category_breadcrumbs(),
    'stock_info' : stock_info(),
    'number_of_offers' : number_of_offers(),
    'price_start_from' : price_start_from(),
    'product_rating' : product_rating(),
    'product_reviews' : product_reviews(),
    'fbt_content' : fbt_content(),
    'images_url' : images_url(),
    'card_offer' : card_offer(),
    'selected_variant' : selected_variant(),
    'all_variants' : all_variants(),
    'product_description' : product_description(),
    'specification' : specification(),
    'additional_info_table' : additional_info_table(),
    'amazon_tags' : amazon_tags(),
    'product_input_url' : product_input_url(url),
    'product_output_url' : product_output_url(),
    'warranty' : warranty()
}]
product_df = pd.DataFrame(product_data)
product_df

,extract_date,product_name,asin,old_price,new_price,seller_info_pp,brand,category_breadcrumbs,stock_info,number_of_offers,...,card_offer,selected_variant,all_variants,product_description,specification,additional_info_table,amazon_tags,product_input_url,product_output_url,warranty
0,2023-04-18 19:48:18,"SAMSUNG Galaxy S22 Ultra Cell Phone, Factory U...",B09MVZSW5V,1199.99,949.99,"{'Ship from': 'Amazon.com', 'Sold by': 'Amazon...",SAMSUNG,Cell Phones & Accessories > Cell Phones,In Stock,14,...,"Pay $79.17/month for 12 months , interest-free...",Phantom Black || 128GB || S22 Ultra,Phantom Black || Burgundy || Green || Phantom ...,"For Gen Z, video isn’t just video. They think ...",{},{'Package Dimensions': '7.05 x 3.66 x 0.98 inc...,Amazon's Choice,https://www.amazon.com/SAMSUNG-Smartphone-Unlo...,https://www.amazon.com/SAMSUNG-Smartphone-Unlo...,


### Xpath Testing

In [ ]:
test_xp = tree.xpath("//div[@id='centerCol']//div[@class='a-section maple-banner__text']//span/text()")
test_xp

## Seller Page Data Scrape

In [32]:
def seller_url_link():
    home_url = "https://www.amazon.com"
    asin_no = asin()
    seller_url =  home_url + "/gp/aod/ajax/ref=auto_load_aod?asin=" + asin_no + f"&pageno="
    # Amazon.ae
    # seller_url =  home_url + "/gp/aod/ajax/ref=auto_load_aod?asin=" + asin_no + f"&pageno="
    return seller_url

seller_url_link()

'https://www.amazon.com/gp/aod/ajax/ref=auto_load_aod?asin=B09MVZSW5V&pageno='

In [33]:
# Seller Page data functions
def seller_name_sp_func(seller_row, seller_tree):
    # no_of_offers + 1 As we get 1 extra xpath in offer, if add not+2
    seller_name_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='aod-offer-soldBy']//a[@class='a-size-small a-link-normal'][1]/text()")
    seller_name_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='aod-offer-soldBy']//span[@class='a-size-small a-color-base'][1]/text()")
    # seller_row-1 : xpath row start from 1. 1st was skip i.e BuyBox
    seller_name_sp_xp3 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='aod-offer-soldBy']//a[@class='a-size-small a-link-normal']/text()")
    # for amazon seller it wont show anchor tag
    seller_name_sp_xp4 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='aod-offer-soldBy']//span[@class='a-size-small a-color-base']/text()")

    seller_name_sp = ''
    if seller_name_sp_xp1 and seller_row == 1:
        seller_name_sp = seller_name_sp_xp1[0].strip()
    elif seller_name_sp_xp2 and seller_row == 1:
        seller_name_sp = seller_name_sp_xp2[0].strip()
    elif seller_name_sp_xp3:
        seller_name_sp = seller_name_sp_xp3[0].strip()
    elif seller_name_sp_xp4:
        seller_name_sp = seller_name_sp_xp4[0].strip()

    return seller_name_sp

def now_price_sp_func(seller_row, seller_tree):
    now_price_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//span[@class='a-price-whole'][1]/text()")
    now_price_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//span[@class='a-price-whole']/text()")

    now_price_sp = ''
    if now_price_sp_xp1 and seller_row == 1:
        now_price_sp = now_price_sp_xp1[0].strip()
    elif now_price_sp_xp2:
        now_price_sp = now_price_sp_xp2[0].strip()
        
    now_price_sp = re.sub('[^.0-9]+', '', now_price_sp)

    return now_price_sp

def seller_rating_sp_func(seller_row, seller_tree):
    seller_rating_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='aod-offer-seller-rating'][1]/child::i/@class")
    seller_rating_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='aod-offer-seller-rating'][1]/child::i/@class")

    seller_rating_sp = ''
    if seller_rating_sp_xp1 and seller_row == 1:
        seller_rating_sp = seller_rating_sp_xp1[0].split("mini-")[1].split()[0].replace("-", ".")
    elif seller_rating_sp_xp2:
        seller_rating_sp = seller_rating_sp_xp2[0].split("mini-")[1].split()[0].replace("-", ".")

    return seller_rating_sp

def shipping_cost_sp_func(seller_row, seller_tree):
    shipping_cost_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE'][1]/child::a/text()")
    shipping_cost_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE'][1]/child::a/text()")

    shipping_cost_sp = ''
    if shipping_cost_sp_xp1 and seller_row == 1:
        shipping_cost_sp = shipping_cost_sp_xp1[0].strip()
        if 'FREE' in shipping_cost_sp.upper():
            shipping_cost_sp = 0
        elif not 'FREE' in shipping_cost_sp.upper():
            shipping_cost_sp = re.sub("[^0-9.]", "", shipping_cost_sp)
            
    elif shipping_cost_sp_xp2:
        shipping_cost_sp = shipping_cost_sp_xp2[0].strip()
        if 'FREE' in shipping_cost_sp.upper():
            shipping_cost_sp = 0
        elif not 'FREE' in shipping_cost_sp.upper():
            shipping_cost_sp = re.sub("[^0-9.]", "", shipping_cost_sp)
    return shipping_cost_sp

def seller_ranking_sp_func(seller_row, seller_tree):
    return seller_row

def seller_review_count_sp_func(seller_row, seller_tree):
    seller_review_count_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//span[@id='aod-asin-reviews-count-title']/text()")
    seller_review_count_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//span[@id='seller-rating-count-{{iter}}']//span[contains(text(), 'rating')]/text()")

    seller_review_count_sp = ''
    if seller_review_count_sp_xp1 and seller_row == 1:
        seller_review_count_sp = seller_review_count_sp_xp1[0].strip()
        seller_review_count_sp = re.sub("[()]", '', seller_review_count_sp)
    elif seller_review_count_sp_xp2:
        seller_review_count_sp = seller_review_count_sp_xp2[0].strip()
        seller_review_count_sp = re.sub("[()]", '', seller_review_count_sp)

    return seller_review_count_sp

def shipping_days_text_sp_func(seller_row, seller_tree):
    shipping_days_text_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE'][1]/child::b/text()")
    shipping_days_text_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE'][1]/child::b/text()")

    shipping_days_text_sp = ''
    if shipping_days_text_sp_xp1 and seller_row == 1:
        shipping_days_text_sp = shipping_days_text_sp_xp1[0].strip()
    elif shipping_days_text_sp_xp2:
        shipping_days_text_sp = shipping_days_text_sp_xp2[0].strip()

    return shipping_days_text_sp

def product_condition_sp_func(seller_row, seller_tree):
    product_condition_sp_xp1 = seller_tree.xpath(f"//div[@id='aod-pinned-offer']//div[@id='aod-offer-heading']//h5/text()")
    product_condition_sp_xp2 = seller_tree.xpath(f"//div[@id='aod-offer'][{seller_row - 1}]//div[@id='aod-offer-heading']//h5/text()")

    product_condition_sp = ''
    if product_condition_sp_xp1 and seller_row == 1: 
        product_condition_sp = product_condition_sp_xp1[0].strip()
    elif product_condition_sp_xp2:
        product_condition_sp = product_condition_sp_xp2[0].strip()

    return product_condition_sp


# Seller_name 2 diff values calculate
# Todo: Delivered by, Fulfilled by, Ships from
# Todo: Sold by
# Todo: prime_saving
# Todo: Fastest delivery Order within 7 hrs and 36 mins
# Tod: seller_positive_rating_txt - 47% positive over the last 12 months47% positive over the last 12 months

In [34]:
def seller_pages():
    seller_page_no = 1
    no_of_offers = number_of_offers()

    if no_of_offers != '':
        no_of_offers = int(no_of_offers)

        seller_page_no = (no_of_offers // 10)
        if no_of_offers % 10 != 0:
            seller_page_no += 1

    return seller_page_no

print("Seller Pages: ", seller_pages())

Seller Pages:  2


In [35]:
def seller_data_crawl(seller_pages, seller_url_link):
    total_seller_pages = seller_pages()
    seller_url = seller_url_link()
    seller_data_sp = []
    seller_price_list_sp = []
    all_seller_pages_resp = ""

    # Seller pages loop
    # Todo: If one of the seller page response fails, then need to try 3 times hit with different proxy.
    for seller_page in range(1, total_seller_pages + 1):
        time.sleep(random.randint(3,5))
        print(f"seller url link{seller_page}: ", seller_url + str(seller_page))
        seller_resp = requests.get(seller_url + str(seller_page), headers=headers)
        
        # Append all seller pages response -> For saving HTML page
        all_seller_pages_resp += seller_resp.text
        
        seller_tree = html.fromstring(seller_resp.text)
        
        # Todo: Delete Test Code
        # if seller_page == 2:
        #     seller_resp.status_code = 503
        # Todo: check if any seller page response is !== 200. Seller order changed - Major issue
        print("Seller page response: ", seller_resp.status_code)
        
        if seller_resp.status_code == 200:
            print(f"Fetching data for seller page {seller_page}")
            no_of_offers = number_of_offers()
            if no_of_offers != '':
                no_of_offers = int(no_of_offers)
                
            

            # Seller row in one page loop
            for seller_row in range(1, no_of_offers + 1):
                seller_name_sp = seller_name_sp_func(seller_row, seller_tree)
                now_price_sp = now_price_sp_func(seller_row, seller_tree)
                seller_rating_sp = seller_rating_sp_func(seller_row, seller_tree)
                shipping_cost_sp = shipping_cost_sp_func(seller_row, seller_tree)
                seller_ranking_sp = seller_ranking_sp_func(seller_row, seller_tree)
                seller_review_count_sp = seller_review_count_sp_func(seller_row, seller_tree)
                shipping_days_text_sp = shipping_days_text_sp_func(seller_row, seller_tree)
                product_condition_sp = product_condition_sp_func(seller_row, seller_tree)
                
                
                if seller_name_sp != "":
                    seller_data_sp.append({
                        'seller_name_sp': seller_name_sp,
                        'now_price_sp': now_price_sp,
                        'seller_rating_sp': seller_rating_sp,
                        'shipping_cost_sp': shipping_cost_sp,
                        'seller_ranking_sp': seller_ranking_sp,
                        'seller_review_count_sp': seller_review_count_sp,
                        'shipping_days_text_sp': shipping_days_text_sp,
                        'product_condition_sp': product_condition_sp,
                    })
           
        # seller resp != 200        
        else:
            pass

    print("Length seller data", len(seller_data_sp))
    return (seller_data_sp, all_seller_pages_resp)

seller_dataframe, all_seller_pages_resp = seller_data_crawl(seller_pages, seller_url_link)

seller url link1:  https://www.amazon.com/gp/aod/ajax/ref=auto_load_aod?asin=B09MVZSW5V&pageno=1
Seller page response:  200
Fetching data for seller page 1
seller url link2:  https://www.amazon.com/gp/aod/ajax/ref=auto_load_aod?asin=B09MVZSW5V&pageno=2
Seller page response:  200
Fetching data for seller page 2
Length seller data 10


### Seller DataFrame

In [36]:
seller_df = pd.DataFrame(seller_dataframe)
seller_df

,seller_name_sp,now_price_sp,seller_rating_sp,shipping_cost_sp,seller_ranking_sp,seller_review_count_sp,shipping_days_text_sp,product_condition_sp
0,Amazon.com,949,,,1,973 ratings,,New
1,thinkproenterprise,699,5,,2,176 ratings,,Used \n - Very Good
2,Lightnin Deals,999,5,,3,7 ratings,,New
3,Extra E,1049,5,,4,169 ratings,,New
4,Jazzer Shack,1099,5,,5,6194 ratings,,New
5,Amazon.com,949,,,1,973 ratings,,New
6,thinkproenterprise,699,5,,2,176 ratings,,Used \n - Very Good
7,Lightnin Deals,999,5,,3,7 ratings,,New
8,Extra E,1049,5,,4,169 ratings,,New
9,Jazzer Shack,1099,5,,5,6194 ratings,,New


### Concat HTML save for testing - Product Page + Seller page

In [ ]:
final_res_text = res.text + all_seller_pages_resp
# Html write response

try:
    with open("html_page1.htm", "w", encoding='utf-8') as file_obj:
        file_obj.write(str(final_res_text))
    print("HTML file saved successfully.")
except Exception as e:
    print("Error: " ,e)
    

In [ ]:
# html read response
try:
    with open("html_page1.htm", "r", encoding="utf-8") as file_obj:
        page = file_obj.read()
    tree = html.fromstring(page)
except Exception as e:
    print("Error: " ,e)

# Testing

In [ ]:
# Seller Code URL Testing
seller_url = seller_url_link()
seller_res = requests.get(seller_url, headers=headers)
seller_tree = html.fromstring(seller_res.content)
seller_res.status_code

## Other functions